In [1]:
import pandas as pd
import pickle 
import os
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

In [ ]:
def count_stations_with_data_from_2001(directory):
    count = 0
    start_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')

    for station_file in os.listdir(directory):
        file_path = os.path.join(directory, station_file)
        data_day = pd.read_csv(file_path, sep='\s+')
        data_day['Date'] = pd.to_datetime(data_day['YYMMMDD'], format='%y%b%d')
        if data_day['Date'].min() <= start_date:
            count += 1

    return count

directory = 'data/GNSS_day'
stations_with_data_from_2001 = count_stations_with_data_from_2001(directory)
print(f"数据至少从2001年1月1日就开始记录的监测站数量: {stations_with_data_from_2001}")

In [68]:
import os
import numpy as np
import pandas as pd

def has_consecutive_nans(series, window=30):
    """
    检查给定序列中是否存在连续的缺失值。

    参数:
    series (pd.Series): 要检查的序列。
    window (int): 连续缺失值的窗口大小。

    返回:
    bool: 如果存在连续的缺失值，则返回 True，否则返回 False。
    """
    return series.isna().rolling(window=window).apply(lambda x: x.sum() >= window).any()

def construct_gnss_csv(directory, start_date, end_date, output_file):
    # 初始化时间范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    data_frames = []

    for station_file in os.listdir(directory):
        file_path = os.path.join(directory, station_file)
        data_day = pd.read_csv(file_path, sep='\s+')
        data_day['Date'] = pd.to_datetime(data_day['YYMMMDD'], format='%y%b%d')
        if data_day['Date'].min() <= start_date:
            # 只保留2001年1月1日之后的数据
            data_day = data_day[data_day['Date'] >= start_date]

            data_day['combined'] = data_day.apply(lambda row: [row['_north(m)'], row['__east(m)'], row['____up(m)'], np.sqrt(row['__east(m)']**2 + row['_north(m)']**2 + row['____up(m)']**2)], axis=1)
            station_name = os.path.splitext(station_file)[0]
            temp_df = pd.DataFrame(data_day['combined'].values, columns=[station_name], index=data_day['Date'])
            temp_df = temp_df.reindex(date_range)
            is_nan = has_consecutive_nans(temp_df[station_name])
            if not is_nan:
                data_frames.append(temp_df)
                print(f"监测站{station_name}的数据已添加。")
            else:
                print(f"监测站{station_name}的数据不完整，已被忽略。")
    df = pd.concat(data_frames, axis=1)
    
    df.to_csv(output_file)

directory = 'data/GNSS_day'
start_date = pd.to_datetime('2001-01-01')
end_date = pd.to_datetime('2024-08-31')
output_file = 'data/gnss_day_2001_onwards.csv'

construct_gnss_csv(directory, start_date, end_date, output_file)
print(f"CSV文件已创建：{output_file}")

True
监测站SCIA的数据已添加。
True
监测站OPCX的数据不完整，已被忽略。
True
监测站CDMT的数据已添加。
True
监测站CLAR的数据不完整，已被忽略。
True
监测站SBCC的数据不完整，已被忽略。
True
监测站VTIS的数据不完整，已被忽略。
True
监测站WMAP的数据已添加。
True
监测站SPMS的数据已添加。
True
监测站PSDM的数据已添加。
True
监测站RDMT的数据不完整，已被忽略。
True
监测站CTMS的数据不完整，已被忽略。
True
监测站COTD的数据不完整，已被忽略。
True
监测站RTHS的数据不完整，已被忽略。
True
监测站LNMT的数据不完整，已被忽略。
True
监测站NOCO的数据不完整，已被忽略。
True
监测站HVYS的数据已添加。
True
监测站HNPS的数据不完整，已被忽略。
True
监测站CGDM的数据不完整，已被忽略。
True
监测站OVLS的数据已添加。
True
监测站TMAP的数据不完整，已被忽略。
True
监测站FMTP的数据不完整，已被忽略。
True
监测站GMPK的数据不完整，已被忽略。
True
监测站CPBN的数据不完整，已被忽略。
True
监测站WOMT的数据不完整，已被忽略。
True
监测站CBHS的数据不完整，已被忽略。
True
监测站IMPS的数据不完整，已被忽略。
True
监测站MAT2的数据不完整，已被忽略。
True
监测站WRHS的数据不完整，已被忽略。
True
监测站LAPC的数据不完整，已被忽略。
True
监测站OAES的数据已添加。
True
监测站VNCO的数据已添加。
True
监测站CNPP的数据不完整，已被忽略。
True
监测站RHCL的数据不完整，已被忽略。
True
监测站RSTP的数据已添加。
True
监测站OXYC的数据不完整，已被忽略。
True
监测站EWPP的数据已添加。
True
监测站BLYT的数据不完整，已被忽略。
True
监测站WIDC的数据不完整，已被忽略。
True
监测站AZU1的数据不完整，已被忽略。
True
监测站NSSS的数据不完整，已被忽略。
True
监测站ISLK的数据不完整，已被忽略。
True
监测站WKPK的数据不完整，已被忽略。
True


In [ ]:
# 存放数据的 GNSS_fiveminute 文件夹里面数据存放的方式是：监测站名字/年份/天数号码.txt，找到所有监测站的时间范围：最早和最晚的时间

def find_time_range(directory):
    earliest_time = None
    latest_time = None

    for station in os.listdir(directory):
        station_path = os.path.join(directory, station)
        if os.path.isdir(station_path):
            for year in os.listdir(station_path):
                year_path = os.path.join(station_path, year)
                if os.path.isdir(year_path):
                    for day_file in os.listdir(year_path):
                        day_path = os.path.join(year_path, day_file)
                        if os.path.isfile(day_path) and day_file.endswith('.txt'):
                            # 提取时间信息
                            date_str = f"{year}-{day_file.split('.')[0]}"
                            date = pd.to_datetime(date_str, format='%Y-%j')
                            
                            # 更新最早和最晚时间
                            if earliest_time is None or date < earliest_time:
                                earliest_time = date
                            if latest_time is None or date > latest_time:
                                latest_time = date

    return earliest_time, latest_time

directory = 'GNSS_fiveminute'
earliest_time, latest_time = find_time_range(directory)
print(f"最早时间: {earliest_time}")
print(f"最晚时间: {latest_time}")

最早时间: 1994-01-26 00:00:00
最晚时间: 2024-08-31 00:00:00


In [3]:
# CSV文件大小估计

earliest_time = pd.to_datetime('1994-01-26 00:00:00')
latest_time = pd.to_datetime('2024-08-31 00:00:00')

# 计算时间点数量
total_minutes = (latest_time - earliest_time).total_seconds() / 60
time_points = total_minutes / 5

print(f"时间点数量: {time_points}")

# 每个时间点的大小（字节）
time_point_size = 20 + 402 * 8

# 总大小（字节）
total_size = time_points * time_point_size

# 转换为MB
total_size_mb = total_size / (1024 * 1024)

print(f"CSV文件大小估计: {total_size_mb:.2f} MB")

时间点数量: 3218400.0
CSV文件大小估计: 9932.27 MB


In [5]:
# 测试监测站数量

def count_stations_before_2001(directory):
    count = 0
    year_threshold = 2000

    for station in os.listdir(directory):
        station_path = os.path.join(directory, station)
        if os.path.isdir(station_path):
            earliest_year = None
            for year in os.listdir(station_path):
                if year.isdigit():
                    year_int = int(year)
                    if earliest_year is None or year_int < earliest_year:
                        earliest_year = year_int
            if earliest_year is not None and earliest_year <= year_threshold:
                count += 1

    return count

directory = 'GNSS_fiveminute'
stations_before_2001 = count_stations_before_2001(directory)
print(f"2000年及以前建站的监测站数量: {stations_before_2001}")

2000年及以前建站的监测站数量: 137


In [8]:
12*24*365*24, 12*24

(2522880, 288)

In [6]:
start_time = pd.to_datetime('2001-01-01 00:00:00')
end_time = pd.to_datetime('2024-08-31 00:00:00')

# 计算时间点数量
total_minutes = (end_time - start_time).total_seconds() / 60
time_points = total_minutes / 5

print(f"时间点数量: {time_points}")

# 每个时间点的大小（字节）
time_point_size = 20 + 137 * 8

# 总大小（字节）
total_size = time_points * time_point_size

# 转换为MB
total_size_mb = total_size / (1024 * 1024)

print(f"CSV文件大小估计: {total_size_mb:.2f} MB")

时间点数量: 2489184.0
CSV文件大小估计: 2649.24 MB
